<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Dask DataFrames

In the last two sections we built computations with dask.delayed and then ran them on a distributed cluster using dask.distributed.  In this section we use Dask.dataframes to build computations for us in the common case of tabuluar computations.  Dask dataframes look and feel like Pandas dataframes but they run on the same infrastructure that powers dask.delayed (indeed many dask.dataframe functions are built using dask.delayed).

In this notebook we use the same stock data as in notebook 1, but now rather than write for loops we let dask.dataframe construct our computations for us.

In [ ]:
import os

import dask
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'stocks', 'GOOG', '*.csv'), parse_dates=['timestamp'])
df

In [ ]:
%time df.head()

In [ ]:
%time df.high.max().compute()

Lets do the same computation we did in the previous section, but now with a few lines of Pandas-ish code

In [ ]:
%%time
%matplotlib inline

high = df.groupby(df.timestamp.dt.round('1d')).high.max()
low = df.groupby(df.timestamp.dt.round('1d')).low.min()
spread = high - low
spread = spread.compute()  # convert from dask.dataframe to pandas
spread.plot(figsize=(10, 5), title='Daily High-Low Spread')

### Persist data in distributed memory

Every time we run an operation like `df.high.max().compute()` we read through our dataset from disk.  This can be slow, especially because we're reading data from CSV.  We usually have two options to make this faster:

1.  Persist relevant data in memory, either on our computer or on a cluster
2.  Use a faster on-disk format, like HDF5 or Parquet

In this section we start by persisting data in memory.  On a single machine this is often done by doing a bit of pre-processing and data reduction with dask dataframe and then `compute`-ing to a Pandas dataframe and using Pandas in the future.  On a distributed cluster we can't use Pandas, but we can ask Dask to persist data in memory with the `dask.persist` function.

    x = x.persist()

or 

    x, y = dask.persist(x, y)

In [ ]:
from dask.distributed import Client, progress
c = Client('localhost:8786')
c

In [ ]:
df = df.persist()
progress(df)

In [ ]:
df.head()

In [ ]:
%time len(df)

### Exercise

Copy-paste the Daily High-Low Spread plot from above.  How much faster is it?  What is taking all of the time?

In [ ]:
%%time
# copy-paste code from Daily High-Low Spread plot above

### Partitions

One Dask.dataframe is composed of several Pandas dataframes.  The organization of these dataframes can significantly impact performance.  In this section we discuss two common factors that commonly impact performance:

1.  The number of Pandas dataframes can affect overhead.  If the dataframes are too small then Dask might spend more time deciding what to do than Pandas spends actually doing it.  Ideally computations should take 100's of milliseconds.

2.  If we know how the dataframes are sorted then certain operations become much faster

### Number of partitions and repartitioning

When we read in our data from CSV files we got one Pandas dataframe for each day.  Look at the metadata below to determine how many partitions we have.  Each "partition" is a Pandas dataframe.

In [ ]:
df

**Question:** Roughtly how large is each partition?

There are a few ways to answer this:

1.  Look at the diagnostic dashboard to see how much memory is being used.  Divide this by the number of partitions.
2.  Use the [.map_partitions()](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions) method along with the Pandas `memory_usage().sum()` function to determine how many bytes each partition consumes.

### Reduce the number of partitions with repartition

We can bring partitions together with the [repartition](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.repartition) method.  Be sure to persist the dataframe afterwards so that we don't do the repartition step over and over again.

In [ ]:
df = ...

### Compare timings

Use the diagnostic dashboard and the `%time` magic to compare the speed of some operations that we did above.  How have things improved?

### Sorted Index column

*This section doesn't have any exercises.  Just follow along.*

Many dataframe operations like loc-indexing, groupby-apply, and joins are *much* faster on a sorted index.  For example, if we want to get data for a particular day of data it *really* helps to know where that day is, otherwise we need to search over all of our data.

The Pandas model gives us a sorted index column.  Dask.dataframe copies this model, and it remembers the min and max values of every partition's index.

By default, our data doesn't have an index.

In [ ]:
df.head()

So if we search for a particular day it takes a while

In [ ]:
%time df[df.timestamp.dt.round('1d') == '2015-05-05'].compute()

However if we set the timestamp column as the index then this operation can be much much faster.

In [ ]:
%%time
df = df.set_index('timestamp')
df

In [ ]:
%time df.loc['2015-05-05'].compute()

Additionally this lets us do traditional Pandas timeseries functionality.

In [ ]:
%%time 
(df.close
   .resample('1d')
   .mean()
   .fillna(method='ffill')
   .compute()
   .plot(figsize=(10, 5)))